# pip installs

In [2]:
!pip install websocket_client

     |████████████████████████████████| 204kB 4.8MB/s 


# Feeding Websocket Data to DataFrame

- Run as long as you want to collect data. Run for 10 minutes to get good amount of data

In [92]:
#from websocket import WebSocketApp
import websocket
import json
from pprint import pprint
import time
import pandas as pd

URL = "wss://ws-feed.pro.coinbase.com"
df = pd.DataFrame()

def on_message(ws, message):
    """Callback executed when a message comes.
    Positional argument:
    message -- The message itself (string)
    """
    msg = json.loads(message)
    pprint(msg)
    global df
    if msg['type'] == 'match':
        df = df.append(msg, ignore_index=True)
      
    #ws.close()
    
def on_open(socket):
    """Callback executed at socket opening.
    Keyword argument:
    socket -- The websocket itself
    """
    params = {
        "type": "subscribe",
        "channels": [{"name": "matches", "product_ids": ["BTC-USD"]}]
    }
    socket.send(json.dumps(params))
    
def main():
    """Main function."""
    ws = websocket.WebSocketApp(URL, on_open=on_open, on_message=on_message)
    ws.run_forever()
      
if __name__ == '__main__':
    main()

{'channels': [{'name': 'matches', 'product_ids': ['BTC-USD']}],
 'type': 'subscriptions'}
{'maker_order_id': 'cf283ff3-03d5-418d-b2c7-0d905ffbd75d',
 'price': '8349.77',
 'product_id': 'BTC-USD',
 'sequence': 11010284401,
 'side': 'buy',
 'size': '0.00280163',
 'taker_order_id': '183a2ed1-5982-41d4-b011-a846fe38f617',
 'time': '2019-10-14T23:49:11.598000Z',
 'trade_id': 75973774,
 'type': 'last_match'}
{'maker_order_id': '025d4822-7b7a-4852-a991-ce4413ac3006',
 'price': '8349.78',
 'product_id': 'BTC-USD',
 'sequence': 11010284523,
 'side': 'sell',
 'size': '0.03135795',
 'taker_order_id': '873205c3-7b8c-48e1-ab48-1128d94a33ce',
 'time': '2019-10-14T23:49:19.287000Z',
 'trade_id': 75973775,
 'type': 'match'}
{'maker_order_id': '66dc93e8-f05c-4c4a-8e92-9ac9d898d400',
 'price': '8349.78',
 'product_id': 'BTC-USD',
 'sequence': 11010284525,
 'side': 'sell',
 'size': '0.08989290',
 'taker_order_id': '873205c3-7b8c-48e1-ab48-1128d94a33ce',
 'time': '2019-10-14T23:49:19.287000Z',
 'trade_id'

# Checking DF

In [93]:
print(df.shape)
df.head()

(977, 10)


,maker_order_id,price,product_id,sequence,side,size,taker_order_id,time,trade_id,type
0,025d4822-7b7a-4852-a991-ce4413ac3006,8349.78,BTC-USD,1.101028e+10,sell,0.03135795,873205c3-7b8c-48e1-ab48-1128d94a33ce,2019-10-14T23:49:19.287000Z,75973775.0,match
1,66dc93e8-f05c-4c4a-8e92-9ac9d898d400,8349.78,BTC-USD,1.101028e+10,sell,0.08989290,873205c3-7b8c-48e1-ab48-1128d94a33ce,2019-10-14T23:49:19.287000Z,75973776.0,match
2,66dc93e8-f05c-4c4a-8e92-9ac9d898d400,8349.78,BTC-USD,1.101028e+10,sell,0.00220579,ab120fd8-5f3e-44bf-b87b-d8ef5dc3f234,2019-10-14T23:49:20.843000Z,75973777.0,match
3,66dc93e8-f05c-4c4a-8e92-9ac9d898d400,8349.78,BTC-USD,1.101028e+10,sell,0.02935461,2239f10f-e762-4c04-9134-8a149c09f771,2019-10-14T23:49:24.338000Z,75973778.0,match
4,66dc93e8-f05c-4c4a-8e92-9ac9d898d400,8349.78,BTC-USD,1.101028e+10,sell,0.02935407,5e3dfb11-a0a5-4f39-b0ae-e63a3c962c1a,2019-10-14T23:49:28.368000Z,75973779.0,match


In [99]:
df['trade_id'].value_counts().sort_values(ascending=True).head()

75974557.0    1
75974550.0    1
75974282.0    1
75974085.0    1
75974512.0    1
Name: trade_id, dtype: int64

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 10 columns):
maker_order_id    977 non-null object
price             977 non-null object
product_id        977 non-null object
sequence          977 non-null float64
side              977 non-null object
size              977 non-null object
taker_order_id    977 non-null object
time              977 non-null object
trade_id          977 non-null float64
type              977 non-null object
dtypes: float64(2), object(8)
memory usage: 76.4+ KB


In [0]:
df1 = df

# Turn DF into OHLCV 5-Minute DF

In [0]:
df1 = df1[['time', 'price', 'size', 'trade_id']]

In [103]:
print(df1.shape)
df1.head()

(977, 4)


,time,price,size,trade_id
0,2019-10-14T23:49:19.287000Z,8349.78,0.03135795,75973775.0
1,2019-10-14T23:49:19.287000Z,8349.78,0.08989290,75973776.0
2,2019-10-14T23:49:20.843000Z,8349.78,0.00220579,75973777.0
3,2019-10-14T23:49:24.338000Z,8349.78,0.02935461,75973778.0
4,2019-10-14T23:49:28.368000Z,8349.78,0.02935407,75973779.0


In [105]:
new_index = pd.DatetimeIndex(df1['time'])

df1['size'] = df['size'].apply(lambda x: float(x))

df1 = df1.set_index(new_index).drop(columns='time')

vol = df1['size'].resample('5Min').sum()

df1['price'] = df1['price'].astype(float)

ohlc5m = df1['price'].resample('5Min').ohlc()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [106]:
ohlc5m

,open,high,low,close
time,,,,
2019-10-14 23:45:00+00:00,8349.78,8349.78,8349.77,8349.78
2019-10-14 23:50:00+00:00,8349.77,8353.71,8349.00,8351.86
2019-10-14 23:55:00+00:00,8351.86,8355.00,8349.06,8355.00
2019-10-15 00:00:00+00:00,8355.00,8376.72,8354.99,8370.73
2019-10-15 00:05:00+00:00,8370.73,8375.93,8366.61,8367.49
2019-10-15 00:10:00+00:00,8365.49,8367.44,8363.56,8363.56


In [0]:
ohlcv5m = pd.concat([ohlc5m, vol], axis=1)

In [108]:
ohlcv5m

,open,high,low,close,size
time,,,,,
2019-10-14 23:45:00+00:00,8349.78,8349.78,8349.77,8349.78,0.684892
2019-10-14 23:50:00+00:00,8349.77,8353.71,8349.00,8351.86,39.882912
2019-10-14 23:55:00+00:00,8351.86,8355.00,8349.06,8355.00,10.759215
2019-10-15 00:00:00+00:00,8355.00,8376.72,8354.99,8370.73,115.560659
2019-10-15 00:05:00+00:00,8370.73,8375.93,8366.61,8367.49,13.898633
2019-10-15 00:10:00+00:00,8365.49,8367.44,8363.56,8363.56,7.068576


# Turn DF into OHLCV 1-Minute DF

In [0]:
df2 = df

In [113]:
df2 = df2[['time', 'price', 'size', 'trade_id']]

new_index2 = pd.DatetimeIndex(df2['time'])

df2['size'] = df2['size'].apply(lambda x: float(x))

df2 = df2.set_index(new_index2).drop(columns='time')

vol2 = df2['size'].resample('1Min').sum()

df2['price'] = df2['price'].astype(float)

ohlc1m = df2['price'].resample('1Min').ohlc()

ohlcv1m = pd.concat([ohlc1m, vol2], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [114]:
ohlcv1m

,open,high,low,close,size
time,,,,,
2019-10-14 23:49:00+00:00,8349.78,8349.78,8349.77,8349.78,0.684892
2019-10-14 23:50:00+00:00,8349.77,8350.00,8349.00,8350.00,29.592302
2019-10-14 23:51:00+00:00,8350.00,8353.71,8349.99,8351.43,2.316077
2019-10-14 23:52:00+00:00,8351.11,8351.97,8350.01,8350.01,2.170936
2019-10-14 23:53:00+00:00,8350.01,8352.00,8350.00,8351.85,0.962053
2019-10-14 23:54:00+00:00,8351.84,8351.88,8349.91,8351.86,4.841544
2019-10-14 23:55:00+00:00,8351.86,8351.86,8351.22,8351.22,0.925179
2019-10-14 23:56:00+00:00,8350.00,8352.00,8349.06,8352.00,3.839488
2019-10-14 23:57:00+00:00,8353.00,8354.49,8353.00,8353.49,0.659856
